# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [33]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import requests
from bs4 import BeautifulSoup



In [34]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [9]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [10]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [16]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [18]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [19]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'proficient page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'}]}

In [20]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [21]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [22]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [23]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [24]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
6 days ago
•
8.71k
•
897
zai-org/GLM-4.7-Flash
Updated
about 12 hours ago
•
15.2k
•
759
google/translategemma-4b-it
Updated
5 days ago
•
35.6k
•
391
Lightricks/LTX-2
Updated
1 day ago
•
1.74M
•
1.22k
kyutai/pocket-tts
Updated
1 day ago
•
31.8k
•
335
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.01k
Qwen Image Multiple Angles 3D Camera
🎥
1.01k
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.65k
Z Image Turbo
🖼
1.65k
Generate stunning images from text descriptions in sec

In [25]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [26]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [27]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 18 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-Image\nUpdated\n6 days ago\n•\n8.71k\n•\n897\nzai-org/GLM-4.7-Flash\nUpdated\nabout 13 hours ago\n•\n15.2k\n•\n759\ngoogle/translategemma-4b-it\nUpdated\n5 days ago\n•\n35.6k\n•\n391\nLightricks/LTX-2\nUpdated\n1 day ago\n•\n1.74M\n•\n1.22k\nkyutai/pocket-tts\nUpdated\n1 day ago\n•\n31.8k\n•\n335\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n1.01k\nQwen Image Multiple

In [28]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [29]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


# Hugging Face

## The AI Community Building the Future

Hugging Face is a leading platform and community empowering the global machine learning (ML) ecosystem to collaborate, innovate, and build the future of AI. Serving as a central hub for ML practitioners, researchers, and enterprises, Hugging Face provides tools, models, datasets, and applications to accelerate development and knowledge sharing in AI.

---

## What We Offer

- **Models & Datasets**  
  Access over 2 million state-of-the-art machine learning models and more than 500,000 datasets. Our resources cover a wide range of modalities including text, images, video, audio, and even 3D data, supporting diverse AI applications.

- **Spaces**  
  Experiment with and deploy ML applications via the Hugging Face Spaces platform, enabling users to quickly try out demos, share their work, and collaborate on projects.

- **Open Source Stack**  
  Empower your innovation with our robust open-source libraries and tools designed to help you build, train, and deploy ML models faster.

- **Enterprise Solutions & Compute**  
  For businesses, we provide scalable, paid compute options and tailored enterprise solutions to integrate advanced AI capabilities safely and efficiently.

---

## Our Community & Culture

Hugging Face is more than a platform—it is a vibrant, fast-growing community of ML engineers, researchers, scientists, and AI enthusiasts dedicated to collaboration and ethical AI development. We foster an inclusive ecosystem where sharing knowledge, building portfolios, and contributing to open source is encouraged and facilitated.

---

## Why Hugging Face?

- A **collaborative platform** that supports limitless public hosting and sharing of models, datasets, and applications.
- **Accelerate development** through shared resources and a thriving community.
- **Multi-modal support** for experiments spanning text, images, audio, video, and 3D.
- Opportunity to **build your professional ML profile** and contribute to breakthrough AI innovations.

---

## Customers & Users

Hugging Face serves a diverse and growing global user base, including:

- AI researchers and academic institutions
- Machine learning engineers and developers
- Startups and tech companies leveraging AI
- Large enterprises seeking reliable AI infrastructure and solutions

---

## Careers at Hugging Face

Join a passionate team at the forefront of AI innovation. Hugging Face offers opportunities for talented individuals eager to shape the future of machine learning. We value collaboration, open source contributions, and ethical AI development.

---

## Get Involved

- Browse or contribute to **2M+ models** and **500k+ datasets**
- Explore over **1M+ AI applications** built by the community
- Sign up to share your work, learn from peers, and accelerate your AI projects  
- Access our [Documentation](https://huggingface.co/docs) and community resources

---

## Contact & Links

- Website: [huggingface.co](https://huggingface.co)  
- Join the Community: Connect with thousands of AI creators globally  
- Enterprise Solutions: Tailored support for your business AI needs  

---

*Hugging Face – Transforming the future of AI by building an open, collaborative, and ethical machine learning ecosystem.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [30]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [31]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is a leading AI community and collaboration platform that is building the future of machine learning (ML). It serves as a vibrant hub where machine learning engineers, scientists, and AI enthusiasts come together to share, explore, and develop state-of-the-art ML models, datasets, and applications. 

With a rapidly growing global community, Hugging Face empowers users to learn, collaborate, and contribute towards building an open, ethical AI ecosystem.

---

## What We Offer

- **Models**: Access and browse over **2 million** open-source ML models covering various modalities including text, image, video, audio, and even 3D.
- **Datasets**: Explore and contribute to a massive repository of **500,000+ datasets** to supercharge ML training and research.
- **Spaces**: Deploy and share your live ML applications and demos easily with the community.
- **Community Collaboration**: A central place to host, share, discover, and experiment together on ML innovations and breakthroughs.
- **Open Source Stack**: Leverage Hugging Face's open-source tools to accelerate your ML projects.
- **Enterprise Solutions**: Access paid compute resources and tailored enterprise-grade solutions for scaling ML initiatives.

---

## Company Culture

At Hugging Face, collaboration and openness are at the core of our culture. We believe in:

- **Building an ethical AI future together** — promoting transparency, inclusivity, and responsibility in AI development.
- **Community-driven innovation** — nurturing a fast-growing community where knowledge sharing is key.
- **Empowering creators** — helping individuals build their ML portfolios and showcase their work globally.
- **Supporting learning and growth** — providing resources and environments for continuous ML education and experimentation.

---

## Customers & Users

Our platform serves a diverse range of users including:

- Machine Learning Engineers and Scientists
- AI Researchers and Academics
- Startups and Enterprises seeking AI transformation
- Developers building AI-powered applications
- Enthusiasts and learners passionate about AI and ML

Organizations leverage Hugging Face for access to cutting-edge models, collaborative tools, and scalable compute solutions.

---

## Careers at Hugging Face

Join us to be at the forefront of AI innovation! Hugging Face offers exciting opportunities for passionate professionals who want to:

- Contribute to open-source ML projects used worldwide.
- Collaborate with a dynamic, diverse, and highly skilled community.
- Drive the future of AI technology in a supportive, inclusive workplace.
- Work on challenging problems spanning datasets, model training, deployment, and more.

Explore careers on our website and become part of a mission-driven company building the future of AI.

---

## Get Started with Hugging Face

- Explore and experiment with millions of ML models and datasets.
- Create and share applications with the community through Spaces.
- Build your professional ML portfolio.
- Access enterprise tools and scalable compute solutions.

**Join the AI community building the future.**

[Sign Up Now](https://huggingface.co/signup) | [Explore Models](https://huggingface.co/models) | [Browse Datasets](https://huggingface.co/datasets) | [Discover Spaces](https://huggingface.co/spaces)

---

### Contact & Social

Visit: [huggingface.co](https://huggingface.co)  
Connect with a vibrant AI community shaping tomorrow’s technology today.

---

*Colors that inspire us:*  
- Yellow Gold: #FFD21E  
- Orange: #FF9D00  
- Gray: #6B7280

*Hugging Face: The Home of Machine Learning Community.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>